In [1]:
import pandas as pd
import time

In [6]:
pathfile = "../processed_data/airline.csv"

df1 = pd.read_csv(pathfile)

In [8]:
# Test uniqueness with an additional column like 'Origin'
unique_rows_with_origin = df1.drop_duplicates(
    subset=['Year', 'Month', 'DayofMonth', 'DepTime', 'FlightNum', 'Origin', 'Dest'])
print(
    f"Total rows: {df1.shape[0]}, , Unique rows by composite index: {unique_rows_with_origin.shape[0]}")

Total rows: 9882798, , Unique rows by composite index: 9882791


In [9]:
# Find rows with duplicate composite keys
duplicate_rows = df1[df1.duplicated(
    subset=['Year', 'Month', 'DayofMonth', 'DepTime', 'FlightNum', 'Origin', 'Dest'], keep=False)]

# Display duplicates for inspection
print(duplicate_rows)

         ActualElapsedTime  AirTime  ArrDelay  ArrTime  CRSArrTime  \
1393779                NaN      NaN       NaN      NaN        1835   
1408195              135.0      NaN      18.0   1348.0        1330   
2331778                NaN      NaN       NaN      NaN        1327   
2899134              196.0    167.0      -9.0   2136.0        2145   
3418508                NaN      NaN       NaN      NaN           0   
3762776                NaN      NaN       NaN      NaN           0   
3996977                NaN      NaN       NaN      NaN        1947   
4077638              186.0    170.0     -14.0   2126.0        2140   
4739939                NaN      NaN       NaN      NaN        1325   
5167899                NaN      NaN       NaN      NaN         743   
7429331              116.0      NaN       4.0   1329.0        1325   
7918780                NaN      NaN       NaN      NaN        2322   
9079057                NaN      NaN       NaN      NaN        1301   
9236868             

In [10]:
# Check for missing values in each column
missing_values = df1.isnull().sum()
# Display only columns with missing values
print(missing_values[missing_values > 0])

ActualElapsedTime     206652
AirTime              3141004
ArrDelay              206652
ArrTime               206399
CRSElapsedTime          2074
CancellationCode     9824178
CarrierDelay         7145066
DepDelay              183923
DepTime               183923
Distance               16155
LateAircraftDelay    7145066
NASDelay             7145066
SecurityDelay        7145066
TailNum              2990853
TaxiIn               2991662
TaxiOut              2990517
WeatherDelay         7145066
dtype: int64


In [12]:
# Drop rows where critical delay columns have missing values
df1 = df1.dropna(subset=['ArrDelay', 'DepDelay', 'CRSElapsedTime', 'Distance'])

In [13]:
# Fill missing values with the median (more robust to outliers) or mean
df1['TaxiIn'].fillna(df1['TaxiIn'].median(), inplace=True)
df1['TaxiOut'].fillna(df1['TaxiOut'].median(), inplace=True)
df1['AirTime'].fillna(df1['AirTime'].mean(), inplace=True)

In [14]:
# Fill missing binary indicators with 0
df1['Cancelled'].fillna(0, inplace=True)
df1['Diverted'].fillna(0, inplace=True)

In [15]:
# Impute actual departure and arrival times with scheduled times if missing
df1['DepTime'].fillna(df1['CRSDepTime'], inplace=True)
df1['ArrTime'].fillna(df1['CRSArrTime'], inplace=True)

In [16]:
# Check for any remaining missing values
print(df1.isnull().sum())

ActualElapsedTime          0
AirTime                    0
ArrDelay                   0
ArrTime                    0
CRSArrTime                 0
CRSDepTime                 0
CRSElapsedTime             0
CancellationCode     9660216
Cancelled                  0
CarrierDelay         6975525
DayOfWeek                  0
DayofMonth                 0
DepDelay                   0
DepTime                    0
Dest                       0
Distance                   0
Diverted                   0
FlightNum                  0
LateAircraftDelay    6975525
Month                      0
NASDelay             6975525
Origin                     0
SecurityDelay        6975525
TailNum              2923093
TaxiIn                     0
TaxiOut                    0
UniqueCarrier              0
WeatherDelay         6975525
Year                       0
dtype: int64


In [17]:
# Use pd.concat to display the first and last three rows
pd.concat([df1.head(3), df1.tail(3)])

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,...,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
0,154.0,122.000000,90.0,1850.0,1720,1455,145.0,NaN,0,0.0,...,6,23.0,ORD,0.0,N293AA,8.0,24.0,AA,0.0,2006
1,159.0,125.000000,59.0,1703.0,1604,1510,114.0,NaN,0,NaN,...,9,NaN,DTW,NaN,N8921E,19.0,15.0,NW,NaN,1997
3,70.0,102.819489,80.0,20.0,2300,2100,60.0,NaN,0,NaN,...,2,NaN,MDW,NaN,NaN,5.0,13.0,ML (1),NaN,1991
9882794,137.0,102.819489,-10.0,708.0,718,550,148.0,NaN,0,NaN,...,8,NaN,MCO,NaN,NaN,5.0,13.0,DL,NaN,1992
9882795,76.0,61.000000,-4.0,736.0,740,620,80.0,NaN,0,NaN,...,5,NaN,ONT,NaN,N639@@,3.0,12.0,WN,NaN,2001
9882796,157.0,124.000000,18.0,3.0,2345,2015,150.0,NaN,0,0.0,...,12,7.0,SEA,0.0,N916FR,6.0,27.0,F9,0.0,2005


In [18]:
df1['CancellationCode'].fillna("Not Canceled", inplace=True)

In [19]:
delay_columns = ['CarrierDelay', 'NASDelay',
                 'LateAircraftDelay', 'WeatherDelay', 'SecurityDelay']
df1[delay_columns] = df1[delay_columns].fillna(0)

In [20]:
# Fill with 'Unknown' for missing TailNum values
df1['TailNum'].fillna("Unknown", inplace=True)

In [21]:
# Check for any remaining missing values
print(df1.isnull().sum())

ActualElapsedTime    0
AirTime              0
ArrDelay             0
ArrTime              0
CRSArrTime           0
CRSDepTime           0
CRSElapsedTime       0
CancellationCode     0
Cancelled            0
CarrierDelay         0
DayOfWeek            0
DayofMonth           0
DepDelay             0
DepTime              0
Dest                 0
Distance             0
Diverted             0
FlightNum            0
LateAircraftDelay    0
Month                0
NASDelay             0
Origin               0
SecurityDelay        0
TailNum              0
TaxiIn               0
TaxiOut              0
UniqueCarrier        0
WeatherDelay         0
Year                 0
dtype: int64


In [22]:
df1.shape

(9660216, 29)

In [23]:
df1.isna().sum()

ActualElapsedTime    0
AirTime              0
ArrDelay             0
ArrTime              0
CRSArrTime           0
CRSDepTime           0
CRSElapsedTime       0
CancellationCode     0
Cancelled            0
CarrierDelay         0
DayOfWeek            0
DayofMonth           0
DepDelay             0
DepTime              0
Dest                 0
Distance             0
Diverted             0
FlightNum            0
LateAircraftDelay    0
Month                0
NASDelay             0
Origin               0
SecurityDelay        0
TailNum              0
TaxiIn               0
TaxiOut              0
UniqueCarrier        0
WeatherDelay         0
Year                 0
dtype: int64

In [24]:
df1.columns

Index(['ActualElapsedTime', 'AirTime', 'ArrDelay', 'ArrTime', 'CRSArrTime',
       'CRSDepTime', 'CRSElapsedTime', 'CancellationCode', 'Cancelled',
       'CarrierDelay', 'DayOfWeek', 'DayofMonth', 'DepDelay', 'DepTime',
       'Dest', 'Distance', 'Diverted', 'FlightNum', 'LateAircraftDelay',
       'Month', 'NASDelay', 'Origin', 'SecurityDelay', 'TailNum', 'TaxiIn',
       'TaxiOut', 'UniqueCarrier', 'WeatherDelay', 'Year'],
      dtype='object')

In [26]:
# Save to CSV (if needed)
output_path = "../processed_data/eda_airline_data.csv"

store = pd.DataFrame(df1)

store.to_csv(output_path, index=False)